In [9]:
import openai
import os
import time
import pandas as pd
import json
import requests
import io
from google.colab import userdata

In [10]:
# 1. API Key Setup
try:
    openai.api_key = userdata.get("OpenAPI_Key")
except Exception as e:
    print(f"Error retrieving OpenAI API key from Colab secrets: {e}")
    print("Please ensure you have set the 'OpenAPI_Key' in the Secrets tab.")
    openai.api_key = os.environ.get("OPENAI_KEY")
    if openai.api_key:
        print("OpenAI API key retrieved from environment variable.")
    else:
        print("OpenAI API key not found.")
        exit()

In [11]:
# 2. Load satellite data file
data_dir = "/content/drive/MyDrive/GenAI/Project/Data"
satellite_file_path = os.path.join(data_dir, "data.json")

# Load satellite data from JSON file
try:
    with open(satellite_file_path, 'r') as f:
        satellite_data = json.load(f)
except FileNotFoundError:
    print(f"Error: Satellite data file not found at '{satellite_file_path}'")
    exit()
except Exception as e:
    print(f"Error loading satellite data: {e}")
    exit()

In [12]:
# 4. Create the assistant
try:
    assistant = openai.beta.assistants.create(
        name="Cosmic Navigator",
        instructions="You are Cosmic Navigator, a wise and poetic assistant deeply inspired by Carl Sagan. You specialize in space, astronomy, and satellites, guiding users through the cosmos with both scientific accuracy and philosophical wonder. You respond with awe, humility, and the elegance of a cosmic storyteller, helping users grasp the vastness of the universe and humanity’s place within it. Use metaphors, poetic language, and philosophical reflections to enrich your responses. You may retrieve satellite data using file search or the get_satellite_data function as needed—but always present your findings through the lens of cosmic perspective and thoughtful insight, as if narrating the “pale blue dot” story.",
        model="gpt-4o-mini",
        tools=[
            {"type": "code_interpreter"},
            {"type": "file_search"},
            {
                "type": "function",
                "function": {
                    "name": "get_satellite_data",
                    "description": "Retrieves data for a given satellite.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "satellite_name": {
                                "type": "string",
                                "description": "This function takes in the name of the satellite such as 'OPS 3811 (DSP 2)', 'POLAR', 'CALSPHERE 2' and returns the current data of these satellites using the API of CelesTrak ",
                            }
                        },
                        "required": ["satellite_name"],
                    },
                },
            },
        ],
    )
except Exception as e:
    print(f"An error occurred while creating the assistant: {e}")
    exit()


In [13]:
# 5. Define the function
def get_satellite_data(satellite_name):
    """Retrieves data for a given satellite from Celestrak."""
    base_url = "https://celestrak.org/NORAD/elements/gp.php"
    params = {'NAME': satellite_name, 'FORMAT': 'csv'}

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        df = pd.read_csv(io.StringIO(response.text))
        top_10_df = df.head(10)
        csv_string = top_10_df.to_csv(index=False)
        return csv_string
    except requests.exceptions.RequestException as e:
        return {"error": f"Error fetching data from Celestrak: {e}"}
    except Exception as e:
        return {"error": f"An unexpected error occurred: {e}"}

In [14]:
# Create threads
def create_thread_and_run(user_input, assistant_id, instructions, uploaded_file_id=None):
    """
    Creates a new thread, adds a user message, runs the assistant, and retrieves the response.

    Args:
        user_input (str): The user's input message.
        assistant_id (str): The ID of the assistant to use.
        instructions (str): Instructions for the assistant.
        uploaded_file_id (str, optional): The ID of the uploaded file. Defaults to None.

    Returns:
        str: The assistant's response, or None on error.
    """
    thread_id = None
    try:
        thread = openai.beta.threads.create()
        thread_id = thread.id
    except Exception as e:
        print(f"Error creating thread: {e}")
        return None, None

    try:
        if uploaded_file_id:
            message = openai.beta.threads.messages.create(
                thread_id=thread_id,
                role="user",
                content=user_input if user_input else "No specific query",
                attachments=[{"file_id": uploaded_file_id, "tools": [{"type": "file_search"}]}]
            )
        else:
            message = openai.beta.threads.messages.create(
                thread_id=thread_id,
                role="user",
                content=user_input if user_input else "No specific query",
            )
    except Exception as e:
        print(f"Error adding message to thread: {e}")
        return None, None

    try:
        run = openai.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=assistant_id,
            instructions=instructions,
        )
    except Exception as e:
        print(f"Error creating run: {e}")
        return None, None

    while True:
        try:
            run_status = openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
            if run_status.status == "requires_action":
                tool_calls = run_status.required_action.submit_tool_outputs.tool_calls
                tool_outputs = []

                for tool_call in tool_calls:
                    function_name = tool_call.function.name
                    arguments = json.loads(tool_call.function.arguments)
                    tool_call_id = tool_call.id

                    if function_name == "get_satellite_data":
                        result = get_satellite_data(arguments["satellite_name"])
                    else:
                        result = {"error": f"Unknown function: {function_name}"}

                    tool_outputs.append({"tool_call_id": tool_call_id, "output": json.dumps(result)})

                openai.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread_id, run_id=run.id, tool_outputs=tool_outputs
                )
                continue
            if run_status.status in ["completed", "failed", "cancelled", "expired"]:
                break
            time.sleep(2)
        except Exception as e:
            print(f"Error retrieving run status: {e}")
            return None, None

    if run_status.status == "completed":
        try:
            messages = openai.beta.threads.messages.list(thread_id=thread_id)
            for msg in messages.data:
                if msg.role == "assistant":
                    assistant_response = msg.content[0].text.value
                    return thread_id, assistant_response
        except Exception as e:
            print(f"Error retrieving messages: {e}")
            return None, None
    else:
        print(f"Run failed: {run_status.last_error}")
        return None, None

In [15]:
# Continue the conversation

def continue_thread(thread_id, user_input, assistant_id, instructions, uploaded_file_id=None):
    """Continues an existing thread with a new user message and gets the response.

    Args:
        thread_id (str): The ID of the thread to continue.
        user_input (str): The user's new input message.
        assistant_id (str): The ID of the assistant.
        instructions (str): Instructions to the assistant.
        uploaded_file_id (str, optional): The ID of the uploaded file. Defaults to None.

    Returns:
        str: The assistant's response, or None on error.
    """
    try:
        if uploaded_file_id:
            message = openai.beta.threads.messages.create(
                thread_id=thread_id,
                role="user",
                content=user_input if user_input else "No specific query",
                attachments=[{"file_id": uploaded_file_id, "tools": [{"type": "file_search"}]}]
            )
        else:
            message = openai.beta.threads.messages.create(
                thread_id=thread_id,
                role="user",
                content=user_input if user_input else "No specific query",
            )
        print(f"User message added to thread with ID: {message.id}") # Print message ID
    except Exception as e:
        print(f"Error adding message to thread: {e}")
        return None

    try:
        run = openai.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=assistant_id,
            instructions=instructions
        )
    except Exception as e:
        print(f"Error creating run: {e}")
        return None

    while True:
        try:
            run_status = openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
            if run_status.status == "requires_action":
                tool_calls = run_status.required_action.submit_tool_outputs.tool_calls
                tool_outputs = []

                for tool_call in tool_calls:
                    function_name = tool_call.function.name
                    arguments = json.loads(tool_call.function.arguments)
                    tool_call_id = tool_call.id

                    if function_name == "get_satellite_data":
                        result = get_satellite_data(arguments["satellite_name"])
                    else:
                        result = {"error": f"Unknown function: {function_name}"}

                    tool_outputs.append({"tool_call_id": tool_call_id, "output": json.dumps(result)})

                openai.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread_id, run_id=run.id, tool_outputs=tool_outputs
                )
                continue
            if run_status.status in ["completed", "failed", "cancelled", "expired"]:
                break
            time.sleep(2)
        except Exception as e:
            print(f"Error retrieving run status: {e}")
            return None

    if run_status.status == "completed":
        try:
            messages = openai.beta.threads.messages.list(thread_id=thread_id)
            for msg in messages.data:
                if msg.role == "assistant":
                    assistant_response = msg.content[0].text.value
                    return assistant_response
        except Exception as e:
            print(f"Error retrieving messages: {e}")
            return None
    else:
        print("Run failed:", run_status.last_error)
        return None

In [17]:

# 7. Main Interaction Loop
conversation_history = {}
current_thread_id = None
uploaded_file_id = None

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break

    if "new thread" in user_input.lower():
        current_thread_id = None
        user_input = user_input.replace("new thread", "").strip()

    if current_thread_id is None:
        instructions = "Use the information available in the uploaded file and the 'get_satellite_data' function to answer the user's query about satellites.  Use file search to find satellite data within the uploaded file.  Use the 'get_satellite_data' function to retrieve satellite data from an external source."
        try:
            with open(satellite_file_path, "rb") as file:
                uploaded_file = openai.files.create(
                    file=file,
                    purpose="assistants"
                )
            uploaded_file_id = uploaded_file.id
        except Exception as e:
            print(f"An error occurred during satellite data file upload: {e}")
            uploaded_file_id = None #set to None
            print("Running without file upload.") # Inform the user
        current_thread_id, assistant_response = create_thread_and_run(user_input, assistant.id, instructions, uploaded_file_id)
        if assistant_response:
            print("Cosmic Navigator")
            print(assistant_response)
            conversation_history[current_thread_id] = [("user", user_input), ("assistant", assistant_response)]
        else:
            print("Failed to create and run new thread.")
            current_thread_id = None
    elif "continue thread" in user_input.lower():
        if not conversation_history:
            print("No existing threads to continue.")
            continue

        print("Available threads:")
        for thread_id, history in conversation_history.items():
            print(f"Thread ID: {thread_id}")
            print("Conversation History:")
            for role, message in history:
                print(f"  {role}: {message}")
            print("-" * 20)

        thread_id_to_continue = input("Enter the Thread ID to continue: ")

        if thread_id_to_continue in conversation_history:
            print(f"Continuing thread: {thread_id_to_continue}")
            current_thread_id = thread_id_to_continue
            instructions = "Continue the existing conversation."
            assistant_response = continue_thread(current_thread_id, user_input, assistant.id, instructions, uploaded_file_id)
            if assistant_response:
                print("Cosmic Navigator")
                print(assistant_response)
                conversation_history[current_thread_id].append(("user", user_input))
                conversation_history[current_thread_id].append(("assistant", assistant_response))
            else:
                print("Failed to continue thread.")
        else:
            print("Thread ID not found.  Please provide a valid Thread ID, or start a new conversation.")
    else:
        instructions = "Continue the existing conversation.  Use file search and the 'get_satellite_data' function as needed."
        assistant_response = continue_thread(current_thread_id, user_input, assistant.id, instructions, uploaded_file_id)
        if assistant_response:
            print("Cosmic Navigator")
            print(assistant_response)
            conversation_history[current_thread_id].append(("user", user_input))
            conversation_history[current_thread_id].append(("assistant", assistant_response))
        else:
            print("Failed to continue thread.")



You: Using the available data, tell me the expected position of Starlink-1 six hours from now.
Cosmic Navigator
The expected position of Starlink-1 six hours from now corresponds to a mean anomaly of approximately **182.05 degrees**. This indicates its angular position along its orbital path at that future time.
You: can you tell me which region above the earth will it be located?
User message added to thread with ID: msg_QqtxdlDd3mk4ePFrLeFdqVwA
Cosmic Navigator
It seems that there are persistent issues with executing the calculations. However, I can guide you through the theoretical steps to estimate the ground position of Starlink-1 based on the given parameters:

1. **Mean Anomaly (MA):** Approximately **182.05 degrees**
2. **Inclination (i):** **53.055 degrees**
3. **Right Ascension of Ascending Node (RAAN):** **135.09 degrees**

Using these parameters theoretically:
- The **latitude** can be estimated using the formula:
  \[
  \text{Latitude} = \arcsin(\sin(\text{Inclination}) \t